#### redo w/ batch
- would like to re-do the "2020-06-20" notebook but w/ the batching approach, to see if any deterioration!!?? 
- That is, the batching approach I had used [here](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-14.md#trying-out-that-model-save)

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [5]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)

# ... actually doing this part jsut to get those labels... 
X_train, X_test, y_train, y_test = train_test_split(X, y)
# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_


In [6]:
def do_train(model, X, y, workdir):

    size = X.shape[0]

    indices = np.random.choice(range(size), size=size, replace=False)
    parts = fu.get_partitions(indices, slice_size=1000)

    prev_model_loc = None
    for i, part in enumerate(parts):
        model.fit(X[part], y[part], xgb_model=prev_model_loc)
        fu.log(workdir, f'[{i}] Done fit')

        prev_model_loc = f'{workdir}/model.xg'
        model.save_model(prev_model_loc)

    return model

In [ ]:
%%time
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')

rng = np.random.RandomState(31337)

kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for (i, (train_index, test_index)) in enumerate(kf.split(X)):    
    # preproc
    (X_transformed,
         one_hot_enc, le,
         y_enc) = pv1.preprocess(X[train_index], y[train_index], 
                             neighborhoods)
    
    xgb_model = xgb.XGBClassifier(objective='multi:softprob'
                                 )# .fit(X_transformed, y_enc, verbose=True)
    xgb_model = do_train(model, X_transformed, y_enc, workdir=workdir)
    fu.log(workdir, f'[{i}] Done fit.')
    
    bundle_loc = f'{workdir}/bundle_{i}.joblib'
    joblib.dump({'model': xgb_model}, bundle_loc)
    #
    X_test_transformed = one_hot_enc.transform(X[test_index])
    actuals = le.transform(y[test_index]); len(actuals)
    
    predictions = xgb_model.predict(X_test_transformed)
    confusion = confusion_matrix(actuals, predictions)
    acc = accuracy_score(actuals, predictions)
    
    y_prob_vec = fu.predict_proba(X_test_transformed, bundle_loc=bundle_loc)
    # xgb_model.predict_proba(X_test_transformed)
    fu.log(workdir, f'[{i}] Done fu.predict_proba')
    
    
    logloss = fu.big_logloss(actuals, y_prob_vec, list(range(len(labels))))
    fu.log(workdir, f'[{i}] Done big_logloss, loss={logloss}.')
                          
    # save full now though
    joblib.dump({'model': xgb_model,
                'metrics': {'confusion': confusion,
                           'validation_logloss': logloss,
                           'validation_acc': acc}}, bundle_loc)
    fu.log(workdir, f'[{i}] dumped bundle to {bundle_loc}')
                             